<a href="https://colab.research.google.com/github/silviootero/Trabajo-de-Programacion-Lineal/blob/main/M%C3%A9todo_Simplex_revisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class Simplex:
    def __init__(self, tableau):
        self.tableau = tableau

    def solve(self):
        # Implementar la lógica para resolver el problema de programación lineal utilizando el método simplex revisado
        pass

    def display_solution(self):
        # Implementar la lógica para mostrar la solución paso a paso
        pass

    


    def graphical_method(self, maxim):
        # Implementar el método gráfico para problemas de dos variables
        form = 0
        i = -1
        j = -1
        if maxim == 0:
          while(form <= resultado):
            i = i + 1
            form = funcion(i, 0)

          form = 0
          while(form <= resultado):
            j = j + 1
            form = funcion(0, j)
          
          i = i - 1
          j = j - 1 
          
          return i, j
        elif maxim == 1:
            return 1
        pass

    def sensitivity_analysis(self):
        # Implementar el análisis de sensibilidad
        pass

# Ejemplo de uso
if __name__ == "__main__":
    resultado = 18
    maxim = 0
    # Crear un objeto Simplex con el tableau del problema
    tableau = np.array([[1, -1, 1, 0, 0, 4],
                        [2,  3, 0, 1, 0, 12],
                        [3,  1, 0, 0, 1, 18],
                        [-4, -3, 0, 0, 0, 0]])  # El tableau debe estar en forma aumentada
    simplex_solver = Simplex(tableau)

    # Resolver el problema de programación lineal
    solution = simplex_solver.solve()

    # Mostrar la solución paso a paso
    simplex_solver.display_solution()

    # Realizar el método gráfico (solo para problemas con dos variables)
    x1, x2 = simplex_solver.graphical_method(maxim)

    # Realizar el análisis de sensibilidad
    simplex_solver.sensitivity_analysis()

    grafico(resultado, x1, x2)
    print(x1, x2)



SyntaxError: invalid syntax (2538087923.py, line 15)

In [ ]:
def funcion(x1, x2):
  return 3 * x1 + 5 * x2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def grafico(resultado, x1, x2):
  a, b, c = 3, 2, 5

  x = np.linspace(0, x1, 100)   

  y = (resultado - a * x) / b
  z = (resultado - a * x) / c
  z2 = ((resultado + 4) - a * x) / c


  # Plot the line
  
  fig, ax = plt.subplots()
  ax.plot(x, y, label=f"{a}x + {b}y = {resultado}")
  ax.plot(x, z, label=f"{a}x + {c}z = {resultado}")
  ax.plot(x, z2, label=f"{a}x + {c}z = {resultado + 4}")
  ax.axvline(4, color='red', linestyle='--', label='x = 4')
  ax.axhline(6, color='red', linestyle='--', label='y = 6')

  plt.xlabel('x')
  plt.ylabel('y')
  plt.title(f"Grafico de {a}x + {b}y = {resultado}")
  plt.grid(True)
  plt.legend()
  plt.show()